In [23]:
import efficientnet.keras as efn
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras.layers import Dense, Activation, Flatten, Dropout
import matplotlib.pyplot as plt

In [24]:
# yes=cats, no=dog
base_dir = 'data/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training positive pictures
train_pos_dir = os.path.join(train_dir, 'yes')

# Directory with our training negative pictures
train_neg_dir = os.path.join(train_dir, 'no')

# Directory with our validation positive pictures
validation_pos_dir = os.path.join(validation_dir, 'yes')

# Directory with our validation negative pictures
validation_neg_dir = os.path.join(validation_dir, 'no')


In [25]:

# Set up matplotlib fig, and size it to fit 4x4 pics
import matplotlib.image as mpimg
nrows = 4
ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)
pic_index = 100
train_pos_fnames = os.listdir( train_pos_dir )
train_neg_fnames = os.listdir( train_neg_dir )


next_pos_pix = [os.path.join(train_pos_dir, fname)
                for fname in train_pos_fnames[ pic_index-8:pic_index]
               ]

next_neg_pix = [os.path.join(train_neg_dir, fname)
                for fname in train_neg_fnames[ pic_index-8:pic_index]
               ]

for i, img_path in enumerate(next_pos_pix+next_neg_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

<Figure size 1152x1152 with 0 Axes>

In [26]:
# Add our data-augmentation parameters to ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 2, class_mode = 'binary', target_size = (224, 224))

validation_generator = test_datagen.flow_from_directory( validation_dir, batch_size = 2, class_mode = 'binary', target_size = (224, 224))



Found 40 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [27]:
base_model = efn.EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

In [28]:
for layer in base_model.layers:
    layer.trainable = False

In [32]:
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
predictions = Dense(1, activation="sigmoid")(x)
model_final = Model(base_model.input, predictions)

In [35]:
model_final.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [36]:
eff_history = model_final.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 10, epochs = 10)


Epoch 1/10


C:\Users\sathy\AppData\Local\Temp\ipykernel_13280\1622796652.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  eff_history = model_final.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 10, epochs = 10)


10/10 [==============================] - 13s 200ms/step - loss: 3.4798 - accuracy: 0.8500 - val_loss: 13.8053 - val_accuracy: 0.6000
Epoch 2/10
10/10 [==============================] - 1s 68ms/step - loss: 12.7290 - accuracy: 0.6500 - val_loss: 8.9018 - val_accuracy: 0.6000